# スポーツニュース記事Crawling

- ニュース記事のCrawlingは一般的なCrawlingとは違ってcrawlerが二つ必要です.
- 記事住所をもらってくる crawler 一つと記事住所をもらってきたときに貰った住所を繰り返す crawler 一つが必要です.
- まず記事のアドレスを持ってくる crawler を作る.
- リンクアドレスは概ねa Tag内部のhref=""という属性に入っている.
- このアドレスを持ってくるには .text を使わない.

In [3]:
# ライブラリインポート
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import codecs

## リンクアドレスCrawlerを製作

- 日付と最後のページを入力すると、該当日付にアップロードされた記事アドレスが確認できる。
- アドレスの場合、すべての経路が書かれていないので、この部分を注意して保存しなければなりません。
- Naver記事の場合、アドレスを持って来たリンクどおりに使用すると移動できません。
- 1. アドレスの前の部分にhttps://sports.news.naver.comを追加でコピーして貼ってくれなきゃいけないし
- 2. リンクアドレス内の;を&に変えなければなりません。

In [4]:
# Chromedriverのセッティング.
driver = webdriver.Chrome("C:/Users/dltjs/Data20190921/day3/chromedriver.exe")

# urlを保存するための空いているリスト.
urls = []

for i in range(1, 15):
    # 入力した日付のページに移動
    driver.get("https://sports.news.naver.com/wfootball/news/index.nhn?page="+str(i)+"&date=20190929&isphoto=N")
    time.sleep(2)
    
    # ソースコードを持ってくる
    source = driver.page_source
    
    # parsing
    s1 = BeautifulSoup(source, "html.parser")

    s_articles = s1.find_all("div", class_="content_area")

    a_tag = s_articles[0].find_all("a", class_="title")

    for item in a_tag:
        atag_url = str(item).split('" onclick')[0].split('href="')[1].replace(";", "&")
        link_url = "https://sports.news.naver.com" + atag_url
        urls.append(link_url)
print(len(urls))
driver.close()

277


### ニュース記事収集Crawler

- urlsに保存されたアドレスを利用して各記事別のページに移動.
- 移動したページにある記事本文をtxtファイルに保存.

In [5]:
# Chromedriverのセッティング
driver = webdriver.Chrome("C:/Users/dltjs/Data20190921/day3/chromedriver.exe")

# textファイルに保存できるように、そしてurlsに収集された記事のアドレス通り巡回できるようにする。
f = codecs.open("C:/Users/dltjs/Data20190921/day4/result6.txt", encoding="utf-8", mode='w')
for url in urls:
    driver.get(url)
    time.sleep(2)
    source = driver.page_source

    s1 = BeautifulSoup(source, "html.parser")

    article = s1.find_all('div', class_='news_end font1 size3')

    f.write(article[0].text.strip()+"\r\n")
    f.write("="*80 + "\r\n")
f.close()
driver.close()